In [4]:
import arcpy
from arcpy.sa import *
import datetime
from pysolar.solar import get_altitude, get_azimuth
import pytz

In [5]:

# 设置环境
arcpy.env.workspace = r"F:\cache_data\tif_file\arcpy\DY.gdb"
arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "0"  # 单线程

In [6]:

input_dem = r"D:\ArcGISProjects\workspace\shbyq\feature_raster_file\features_data_dy.gdb\DEM"
latitude = 26.171786  # 都匀的纬度
longitude = 107.482254  # 都匀的经度
calculation_date = datetime.date(2023, 11, 22)

In [15]:
def calculate_average_hillshade(dem, lat, lon, date):
    timezone = pytz.timezone('Asia/Shanghai')
    hillshades = []
    
    for hour in range(8, 17):
        time = datetime.datetime.combine(date, datetime.time(hour, 0))
        time_utc = timezone.localize(time).astimezone(pytz.UTC)
        
        altitude = get_altitude(lat, lon, time_utc)
        azimuth = get_azimuth(lat, lon, time_utc)
        
        if altitude > 0:
            hillshade = Hillshade(dem, azimuth, altitude, "NO_SHADOWS", 1)
            hillshades.append(hillshade)
    
    if hillshades:
        avg_hillshade = CellStatistics(hillshades, "MEAN")
        avg_hillshade.save("average_hillshade")
    else:
        print("警告：在指定的时间范围内没有有效的日照。")

def calculate_convergence_index(aspect):
    def relative_aspect(direction):
        diff = Abs(aspect - direction)
        return Con(diff > 180, 360 - diff, diff)
    
    rel_aspects = [relative_aspect(i * 45) for i in range(8)]
    avg_rel_aspect = CellStatistics(rel_aspects, "MEAN")
    return 90 - avg_rel_aspect

def calculate_closed_depressions(dem):
    filled_dem = Fill(dem)
    return filled_dem - dem

def calculate_twi_from_dem(dem_raster, transmissivity_raster=None, area_conversion=0, method=0):
    slope = Slope(dem_raster, "DEGREE")
    flow_dir = FlowDirection(dem_raster)
    sca = FlowAccumulation(flow_dir)
    tan_beta = Tan(slope)
    
    if area_conversion == 1:
        desc = arcpy.Describe(dem_raster)
        cell_size = desc.meanCellWidth
        sca = sca / cell_size
    
    if method == 0:
        if transmissivity_raster:
            transmissivity = Raster(transmissivity_raster)
            twi = Ln(sca / (transmissivity * tan_beta))
        else:
            twi = Ln(sca / tan_beta)
    elif method == 1:
        if transmissivity_raster:
            transmissivity = Raster(transmissivity_raster)
            twi = Ln(sca / (transmissivity * tan_beta))
        else:
            twi = Ln(sca / tan_beta)
    
    twi.save("twi")
    return twi
def terrain_analysis(dem):
    """
    执行所有地形分析任务
    """
    print("开始地形分析...")
    
    # 加载DEM
    dem_raster = Raster(dem)
    desc = arcpy.Describe(dem_raster)
    cell_size = desc.meanCellWidth
    
    # 基本地形参数
    print("计算坡度和坡向...")
    slope = Slope(dem_raster, "DEGREE")
    slope.save("slope")
    aspect = Aspect(dem_raster)
    aspect.save("aspect")
    
    print("计算曲率...")
    # 正确使用 Curvature 函数
    curvature = Curvature(dem_raster, 1, "profile_curvature", "plan_curvature")
    curvature.save("curvature")
    
    print("计算平均山体阴影...")
    calculate_average_hillshade(dem_raster, latitude, longitude, calculation_date)
    
    print("计算汇聚指数...")
    convergence_index = calculate_convergence_index(aspect)
    
    print("计算封闭洼地...")
    depressions = calculate_closed_depressions(dem_raster)
    
    # 水文分析
    print("执行水文分析...")
    flow_dir = FlowDirection(dem_raster)
    flow_acc = FlowAccumulation(flow_dir)
    
    catchment_area = flow_acc * cell_size * cell_size
    catchment_area.save("catchment_area")
    
    print("计算湿度指数...")
    wetness_index = Ln((flow_acc + 1) * cell_size / (Sin(slope * 0.01745) + 0.001))
    wetness_index.save("wetness_index")
    
    print("计算TWI...")
    calculate_twi_from_dem(dem)
    
    print("计算LS因子...")
    ls_factor = Power(flow_acc * cell_size / 22.13, 0.6) * Power(Sin(slope * 0.01745) / 0.0896, 1.3)
    ls_factor.save("ls_factor")
    
    print("计算通道网络...")
    channel_network = Con(flow_acc > 1000, 1)
    channel_network.save("channel_network")
    
    print("计算流域...")
    basins = Basin(flow_dir)
    basins.save("drainage_basins")
    
    print("计算通道网络基准面...")
    chnl_base = Con(IsNull(channel_network), dem_raster)
    chnl_base.save("channel_network_base_level")
    
    print("计算到通道网络的距离...")
    chnl_dist = EucDistance(channel_network)
    chnl_dist.save("channel_network_distance")
    
    print("计算谷深...")
    valley_depth = dem_raster - chnl_base
    valley_depth.save("valley_depth")
    
    print("计算相对坡位...")
    neighborhood = NbrRectangle(5, 5, "CELL")
    min_elev = FocalStatistics(dem_raster, neighborhood, "MINIMUM")
    max_elev = FocalStatistics(dem_raster, neighborhood, "MAXIMUM")
    rel_slope_pos = (dem_raster - min_elev) / (max_elev - min_elev)
    rel_slope_pos.save("relative_slope_position")
    print("地形分析完成。")

In [16]:
terrain_analysis(input_dem)

开始地形分析...
计算坡度和坡向...
计算曲率...
计算平均山体阴影...
计算汇聚指数...
计算封闭洼地...
执行水文分析...
计算湿度指数...
计算TWI...
计算LS因子...
计算通道网络...
计算流域...
计算通道网络基准面...
计算到通道网络的距离...
计算谷深...
计算相对坡位...
地形分析完成。
